In [1]:
import requests
from bs4 import BeautifulSoup
from lst_py import lst_urls
from tqdm import tqdm
import pandas as pd

In [2]:
lst_g = []
lst_c = []
lst_err = []

In [3]:

# for u in range(len(lst_urls)):
for u in tqdm(range(len(lst_urls))):

    try:
        # URL que deseas extraer
        url = lst_urls[u]

        # Realizar la solicitud HTTP para obtener el contenido de la página
        response = requests.get(url)

        if response.status_code == 200:

            url_data = {}
            url_conse = {}

            url_data["url"] = url
            

            soup = BeautifulSoup(response.text, 'html.parser')

            # FECHAS
            # ### tbl_dates = soup.select("table.table.table_summary")[n_tables[len(tbl_dates)]].select_one("tbody").select("tr")
            tbl_dates = soup.select("table.table.table_summary")
            
            l = None

            
            for i in range(len(tbl_dates)):
                if "Fecha de vencimiento" in f'{tbl_dates[i].encode_contents()}':
                    l = i
            
            tbl_dates = tbl_dates[l].select_one("tbody").select("tr")
            
            
            for i in range(len(tbl_dates)):
                index = 0 if i == 0 else 1
                key = "duracion_years" if i == 0 else tbl_dates[i].select("td")[0].text.strip().replace(" ","_")
                value =  tbl_dates[i].select("td")[index].text.strip().replace(" ","_").strip()
                value = "SIN INFORMACIÓN" if len(value) == 0 else value
                url_data[key] = value
            # -------------------------------------------------------
            # CONSECIONES
            tbl_eml = soup.find("table", {"id": "tblCoberturas"})
            
            if tbl_eml:
                thead = tbl_eml.select("thead")[0].select("tr")[0].select("th")
                tbody = tbl_eml.select("tbody")[0].select("tr")
                for tr in range(len(tbody)):
                    html_tr = tbody[tr].select("td")
                    l_html_tr = len(html_tr)
                    temp_dict = {}
                    for l in range(l_html_tr):
                        e = thead[l].text
                        d = html_tr[l].text.strip()
                        d = "SIN INFORMACIÓN" if len(d) == 0 else d
                        temp_dict["url"] = url
                        temp_dict[e] = d

                    lst_c.append(temp_dict)
            
            # FRECUENCIA PRINCIPAL
            # tbl_frec = soup.find("table", {"id": "tblCoberturas"})
              
            lst_g.append(url_data)
            lst_c.append(url_conse)
        
    except Exception:
        lst_err.append(url)

100%|██████████| 9/9 [00:07<00:00,  1.16it/s]


In [4]:
df_f = pd.DataFrame(lst_g)
df_f.to_csv('./resultados_csv/fechas.csv', index=False)

df_c = pd.DataFrame(lst_c)
df_c.to_csv('./resultados_csv/cobertura.csv', index=False)

df_e = pd.DataFrame(lst_err, columns=["url_con_error"])
df_e.to_csv("./resultados_csv/errores_urls.csv", index=False)